In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from scipy.stats import linregress
import seaborn as sns
import math
from fitter import Fitter

In [ ]:
#olympics_df = pd.read_csv('./archive/Summer-Olympic-medals-1976-to-2008.csv', encoding='latin-1')
olympics_df = pd.read_csv('./archive/athlete_events.csv', encoding='latin-1')
olympics_df.dropna(inplace=True)
olympics_df.drop_duplicates()
olympics_df.head(10)

In [ ]:
countries = olympics_df['Team'].unique()
countries[:10]

In [ ]:
metals = ['Gold', 'Silver', 'Bronze']

nation = []
average_mean = []
trend = []
d_inters = []
std_medals = []
volatility_medals = []
initial_prices = []

for country in countries:
    countryMetals = []
    countryAllMetals = []
    years = []
    country_df = olympics_df[olympics_df['Team'] == country]
    times = country_df['Year'].unique()
    times.sort()
    for time in times:
        years.append(time)
        curYear = country_df[country_df['Year'] == time]
        totalMetals = curYear['Medal'].count()
        countryAllMetals.append(totalMetals)
        golds = curYear[curYear['Medal'] == 'Gold']['Medal'].count()
        silver = curYear[curYear['Medal'] == 'Silver']['Medal'].count()
        bronze = curYear[curYear['Medal'] == 'Bronze']['Medal'].count()
        countryMetals.append([golds,silver, bronze])
    
    if len(countryAllMetals) > 25 and len(years) > 0:
        if years[-1] >= 2008:
            std_dev = np.std(countryAllMetals)
            print("Total mean of Country medals:", sum(countryAllMetals)/len(countryAllMetals))
            print("Total STD of Country medals:", std_dev)
            plt.figure(figsize=(8,4))
            plt.plot(years, countryAllMetals, marker='o')
            plt.xlabel('Year')
            plt.ylabel("Total Medals Earned")
            plt.title(f"Total metals earned by {country} for Summer Olympics")
            plt.show()

            plt.figure(figsize=(8,4))
            plt.plot(years, countryMetals, label=metals)
            plt.legend()
            plt.xlabel('Year')
            plt.ylabel("Total Medals Earned")
            plt.title(f"Total metals earned by {country} for Summer Olympics")
            plt.show()

            plt.figure(figsize=(8,4))
            sns.regplot(x=years, y=countryAllMetals)
            plt.legend()
            plt.xlabel('Year')
            plt.ylabel("Total Medals Earned")
            plt.title(f"Total metals earned by {country} for Summer Olympics")
            plt.show()

            data_med = pd.DataFrame({'year':years, 'Medals':countryAllMetals})
            slope, intercept, r, p, err = linregress(data_med['year'], data_med['Medals'])
            print('Slope of Regression Line', slope)

            #Calculating Volatility
            offset = data_med['Medals'] - (slope * data_med['year'] + intercept)
            volatility = np.std(offset)
            print('Volatility!', volatility, "\n\n")

            nation.append(country)
            average_mean.append(sum(countryAllMetals)/len(countryAllMetals))
            trend.append(slope)
            d_inters.append(intercept)
            std_medals.append(std_dev)
            volatility_medals.append(volatility)
            initial_prices.append(countryAllMetals[-1])

            

## Creating predictions for data above based on the data above


In [ ]:
print(nation)
print(average_mean)
print(trend)
print(d_inters)
print(std_medals)
print(volatility_medals)
print(initial_prices)

## Running a Monte Carlo Prediction on earnings

Reusing the basic shape for the code below but have the variables associated with the work come from the discovered data above. 

In [ ]:


class European_Call_Payoff:

    def __init__(self, strike):
        self.strike = strike

    def get_payoff(self, stock_price):
        if stock_price > self.strike:
            return stock_price - self.strike
        else:
            return 0
# The use of the Brownian motion with the normal distribution made more sense 
class GeometricBrownianMotion:

    def simulate_paths(self):
        while(self.T - self.dt > 0):
            dWt = np.random.normal(0, math.sqrt(self.volatility))  # Brownian motion
            dYt = self.drift*self.current_price*self.dt + self.volatility* self.current_price *dWt  # Change in price (UPDATED)
            self.current_price += dYt  # Add the change to the current price in a logarithmic distribution of growth based on the distributions in the first part.
            self.prices.append(self.current_price)  # Append new price to series
            '''I wanted to add the concept of growing with capping points as this exists in other datasets outside of stocks
            since stocks are usually allowed to grow without bounding
            
            Additionally since the data has both the summer and winter olympics the nummber of metals that can be won swing with the season
            Summer awards a total of 339 Gold medals (depending)
            Winter awards a messley 109 Gold medals (depending)

            From the countries selected above there are 5 that fall into winter strengths and 5 that fall into Summer
            Canada, Sweeden, Finland, Norway and Switzerland 
            
            I was okay leaving volitility unchanged as all I want to mitigate is the natural incline of the system
            '''
            if (self.winterStrong and self.summer) or (self.summer == False and self.winterStrong == False):
                if self.current_price > 40:
                    spin = random.randint(0,3)
                    if spin == 0:
                        self.drift = (.05*self.drift)
                    elif spin == 1:
                        self.drift = (.005*self.drift)
                    else:
                        if self.drift > 0:
                            self.drift = (-.4*self.drift)
                        elif self.drift <= 0:
                            self.drift -= .2
                elif self.current_price > self.average_mean + 2:
                    self.drift -= (.2*self.drift)
                elif self.current_price < self.average_mean - 2:
                    self.drift += (.1*self.drift)
                elif self.current_price < self.average_mean/2:
                    self.drift += .2
            else:
                if self.current_price > 70:
                    spin = random.randint(0,3)
                    if spin == 0:
                        self.drift = (.05*self.drift)
                    elif spin == 1:
                        self.drift = (.005*self.drift)
                    else:
                        if self.drift > 0:
                            self.drift = (-.5*self.drift)
                        elif self.drift <= 0:
                            self.drift -= .3
                elif self.current_price > self.average_mean + 5:
                    self.drift -= (.5*self.drift)
                elif self.current_price < self.average_mean - 5:
                    self.drift += (.25*self.drift)
                elif self.current_price < self.average_mean/2:
                    self.drift += .5
            '''
            End Modification
            '''
            self.T -= self.dt  # Account for the step in time
            self.day += 1

    def __init__(self, initial_price, drift, volatility, average_mean, dt, T, winterStrong, summer):
        self.current_price = initial_price
        self.initial_price = initial_price
        self.drift = drift
        self.volatility = volatility
        self.average_mean = average_mean
        self.dt = dt
        self.T = T
        self.winterStrong = winterStrong
        self.summer = summer
        self.day = 0
        self.prices = [self.current_price]
        self.simulate_paths()

def organizer(paths, initial_price, drift, volatility, average_mean, dt, T, nat, winterStrong, summer):
    price_paths = []

    for i in range(0, paths):
        price_paths.append(GeometricBrownianMotion(initial_price, drift, volatility, average_mean, dt, T, winterStrong, summer).prices)

    call_payoffs = []
    final_prices = []
    #Changed this part of the code to follow the slides MCMC given in 8-11 
    #However there is some confusion as it says that we would need the averages of all of the stocks but we haven't even begun generating Stock a results
    #Because of this I followed the starter code and simply set Sa to 100 and then made the modifications that would be needed for part twos basket question
   
    
    ec = European_Call_Payoff(initial_price)
    
    
    risk_free_rate = 0 # Wasn't defined in the problem description so we will assume 0 for this assignment as I didn't see how to calculate it in the lecture notes
    for price_path in price_paths:
        call_payoffs.append(ec.get_payoff(price_path[-1])/(1 + risk_free_rate))  # We get the last stock price in the series generated to determine the payoff and discount it by one year
        final_prices.append(price_path[-1])

    # Plot the set of generated sample paths
    for price_path in price_paths:
        plt.plot(price_path)
    plt.xlabel('olympic games')
    plt.ylabel('Medals Won')
    plt.title(f"Simulations of Medals won by {nat} Based on Geometric Brownian Motion")
    plt.show()

    print(f"The initial won medals for the {nat} was {initial_price}")
    print(f"Average medal won after {int(1 / dt) * T} olympics: ", round(np.average(final_prices),3))
    print(f'That is a shift of {round(np.average(final_prices) - initial_price, 2)} medals')
    print(f'In other words a/an {round(((np.average(final_prices) - initial_price)*100)/initial_price, 2)}% change in the medals won over the simulated games')

    print("Value in investing", np.average(call_payoffs))


In [ ]:
for ind in range(len(nation)):
    winterStrong = False
    summer = True
    if nation[ind] in ['Canada', 'Sweden', 'Finland', 'Norway', 'Switzerland']:
        winterStrong = True
        print('tru')
    paths = 2000
    initial_pr= initial_prices[ind]
    drift_var = trend[ind]
    volatility = volatility_medals[ind]/100
    aver_meds = average_mean[ind]
    nat = nation[ind]
    #Transforming definition of dt to go over 8 future olympic games. 
    dt = 1/8
    T = 1

    organizer(paths, initial_pr, drift_var, volatility, aver_meds, dt, T, nat, winterStrong, summer)

for ind in range(len(nation)):
    winterStrong = False
    summer = False
    if nation[ind] in ['Canada', 'Sweden', 'Finland', 'Norway', 'Switzerland']:
        winterStrong = True
        print('tru')
    paths = 2000
    initial_pr= initial_prices[ind-1]
    drift_var = trend[ind-1]
    volatility = volatility_medals[ind]/100
    aver_meds = average_mean[ind]
    nat = nation[ind]
    #Transforming definition of dt to go over 8 future olympic games. 
    dt = 1/8
    T = 1

    organizer(paths, initial_pr, drift_var, volatility, aver_meds, dt, T, nat, winterStrong, summer)

# PART 2 (FAIL and Lessons Learned)


### Making Team Based Bandits:: Experimentation and expantion of multi arm Bandit Concept

#### Finland was marked as a declining country with a lot of participation in the Olympic games. 

Having done the above code I wanted to continue my exploration on the maximization concepts. In my proposal I said that my imaginary company could make a country 'gaurentee' their total won golds goes up. I will say that the sent trainer is able to turn 10% of bronze medalists into gold metalists. The goal here is to find the three worst teams and send in this aid. 

In [ ]:
finland_df = olympics_df[olympics_df['Team'] == 'Finland']
finland_df

In [ ]:
fin_sports = finland_df['Sport'].unique()
fin_sports

In [ ]:
for sport in fin_sports:
    sportMedals = []
    sportAllMedals = []
    years = []
    sport_df = finland_df[finland_df['Sport'] == sport]
    times = sport_df['Year'].unique()
    times.sort()
    for time in times:
        years.append(time)
        curYear = sport_df[sport_df['Year'] == time]
        countMedals = curYear['Medal'].count()
        sportAllMedals.append(countMedals)
        golds = curYear[curYear['Medal'] == 'Gold']['Medal'].count()
        silver = curYear[curYear['Medal'] == 'Silver']['Medal'].count()
        bronze = curYear[curYear['Medal'] == 'Bronze']['Medal'].count()
        sportMedals.append([golds,silver, bronze])
    if len(sportAllMedals) > 5:
        f = Fitter(sportMedals,
           distributions=['gamma',
                          'lognorm',
                          'expon',
                          'pareto',
                          "beta",
                          "burr",
                          "norm"])
        f.fit()
        f.summary()
        
        print("Gamma shape: ", f.fitted_param['gamma'][0])
        print("Gamma size: ", f.fitted_param['gamma'][1])
        print("Gamma scale: ", f.fitted_param['gamma'][2])
        
        print("beta a: ", f.fitted_param['beta'][0])
        print("beta b: ", f.fitted_param['beta'][1])
        
        print("pareto a: ", f.fitted_param['pareto'][0])
        print("pareto b: ", f.fitted_param['pareto'][1])
        

        plt.figure(figsize=(5,3))
        sns.histplot(sportAllMedals, bins=40, kde=True)
        plt.xlabel("Medals Won")
        plt.ylabel("Number of Days that Share Difference")
        plt.title(f"Distribution {sport}")
        plt.show()

        
        plt.show()

##### Breakdown: The fitter couldn't discover good models to replicate the data distributions. As I have whittled down the data set it has become spare in data entries thus increaseing the model uncertainty and error. It's like the discussion we had about insurance and making the population size smaller, I just hadn't expected to run into it here!

In [ ]:
def runTeams():
    teams = [
        np.random.gamma(0.38745422015753395, scale = 1.5408578646247686) + 1, # Atletics Team 1
        np.random.gamma(0.18772898880522898, scale = 1.96161488771545) + 1, # Gymnastics  2
        np.random.gamma(0.17334963622150462, scale = 0.4732999223286112), #Speed Skating 3
        np.random.gamma(0.1498571434315908, scale=11.737291520871757), #Ice Hockey 4
        np.random.gamma(0.5610001546976857, scale= 0.8281272674206457), # Wrestling 5
        np.random.gamma(0.39019682458504895, scale=1.6533872843583985), #Rowing 6
        np.random.gamma(0.13958966752650553, scale=0.3515870918747629), #Shooting 7
        np.random.beta(0.5293087828337939, 113.66620224031217), #Boxing 8
        #np.random.pareto(0.5435154352716415), #XCSkiing 9 >> Way to volatile to be useable :( and Burr Broke 
        np.random.beta(0.6173740667197468, 4.8238161078034665), #XCSkiing 9
        np.random.gamma(0.25166399745472745, scale=1.0493386743397677) #Nordic Combined 10
    ]
    return teams

In [ ]:
seasons = 100
teamsOlym = 10
locationOfTrainers = [20,20] # Max of nine
'''If you want results without the trainers then set the locked values to 1 to start'''
locked = [0, 0] # Locked = 1


In [ ]:
def epsilon_greedy(epsilon=.7, iterations=100, lock=False):
    team_results = runTeams()
    # mine results used to find expected means
    team_prods = []
    expected_means = []
    # A history of all the mean gains changes for the given mine
    plot_means = []

    # Shaping the lists
    for olyTmID in range(teamsOlym):
        # Holds all results from calling our mines
        team_prods.append([0])
        # Holds one mean for each mine
        expected_means.append(0)
        # Holds the history of means # Practically 0 I wanted to test how much means changed by but if it's 0 then the system breaks
        plot_means.append([team_results[olyTmID]])
    
    '''
    When To drop? 
    After the 30 mark is when everything levels out if you want to see for yourself comment out the spot where the plot mean drops the first 30 entries :)
    '''
    for i in range(iterations):

        '''
        If you comment this you see where everything reached it's convergence
        '''
        if i == 31:
            for en in range(len(plot_means)):
                plot_means[en] = plot_means[en][30:]
     

        for teamInd in range(teamsOlym):
            '''
            I am making it so that the team that are using the facilities and trainers
            We'll pretend like our promises hold true that our trainers and facililities produce a 20% increase to whichever team is using the facilities with a max of 2 teams able to use them at any time
            Otherwise they follow their normal trends
            '''
            if np.random.random() < epsilon:
                chosen_team = teamInd
                #Doesn't automatically give up the training but does give up the lock that it held. If they're lucky they can get it again but it will still remain unlocked
                if teamInd in locationOfTrainers:
                  
                    if team_results[chosen_team] <= .05:
                        team_results[chosen_team] = .2
                    else:
                        team_results[chosen_team] += team_results[chosen_team] * .2
                    training = locationOfTrainers.index(teamInd)
                    locked[training] = 0
                #If there was an unlocked spot claim it for the next round
                elif 0 in locked:
                    training = locked.index(0)
                    locationOfTrainers[training] = teamInd
                    locked[training] = 1

            else:
                # Simply running itself without having any boost from the training
                chosen_team = teamInd
                # Secure a lock for the worst performing team around
                if 0 in locked:
                    training = locked.index(0)
                    worstTeam = expected_means.index(min(expected_means))
                    locationOfTrainers[training] = worstTeam
                    locked[training] = 1
                    
            # Get the value generated by our selected performer 
            team_reward = team_results[chosen_team]
            # Add it to the history of the performer results
            team_prods[chosen_team].append(team_reward)
            # Update mean
            chosen_mean = sum(team_prods[chosen_team][1:])/len(team_prods[chosen_team])

            expected_means[chosen_team] = chosen_mean
            plot_means[chosen_team].append(chosen_mean)
            # Get new values from our mines
            team_results = runTeams()

    # Plot our graphs
    plt.figure(figsize=(14, 8))

    for machine in range(len(expected_means)):
        plt.plot(range(len(plot_means[machine])),plot_means[machine], label=f"Mine {machine + 1}")
    plt.title(f"Ploted mine mean production according to epsilon = {epsilon}")
    plt.xlabel("Iterations mine was Chosen")
    plt.ylabel("Mean Production Score for given mine")
    plt.legend()
    plt.show()

    '''Unfunctional'''
    # percentGains = 0
    # for avs in plot_means:
    #     change = avs[-1] - avs[0]
    #     percentchange = round((change*100)/avs[0],2)
    #     percentGains += percentchange
    #     print("Percent Changes For teams:", percentchange)
    # print("Average Percent Change", percentchange/len(plot_means))


epsilon_greedy()

# Problem 3: The Greatest game of All! debt!

#### When I had talked with Doctor mario In class I had proposed simulating running events as a game theory for the olympics. However he didn't see it making sense. After a lot of thinking I couldn't come up with anything else and just had to make this. It is still cool! but not part of the theme of olympic project. 

## Despite saying that this is a game it is actually a scheduling problem Solution

Rules of the game:
The game will run for a simulated life of 30 years.
Everyplayer will make the following purchases
- House Payment 
- Car Purchase 

Players or people will have the following as optional items which can occur multiple times.

- Medical Debt
- Credit card Debt
- Personal loans
- more cars


30 different types of player will be used. They will run simulations 1000 times on each style of 
First Come First Serve (FCFS), 2) Shortest-Job-First (SJF) Scheduling, 3) Shortest Remaining Time, 4) Priority Scheduling, and 5) Round Robin Scheduling
This will allow me to understand which style of bill paying results in the most effective payoff system. 

Additionally the items will have the following tagging information:
- Total cost 
- Initial Down Payment
- Intrest Rate
- Payment due

The players themselves will get the following attributes:
- Education
- Income
- Age
- Risk More Debt



Big Purchase Risk Chart

- hs - 4
- a - 3
- b - 2
- ms - 1
- dr - .5

age

- 18-27 - .02
- 28-35 - .03
- 36-45 - .01
- 45+ - .005



## Wanted to implement more on the risk chart but didn't have the time.

In [ ]:
def genNewDebt(debtor):
    # Pick type of debt and threshold
    debtType = random.randint(0, 3)
    risk = random.random()
    debtObj = None
    id = random.randint(1,100)
    # Add A Car 
    if debtType == 0:
        if debtor[2] > risk:
            cost = random.randint(10000, 50000) + (1000 * debtor[3])
            initial_dp = random.randint(0,5000)
            interest_rate = random.randint(5,20) - (initial_dp / 1000)
            monthlyPayment = cost / random.choice([24, 36, 48])
            monthlyPayment += monthlyPayment * (interest_rate/100)
                                                                   
            debtObj = {f"Car{id}": [cost, initial_dp, round(interest_rate,2), round(monthlyPayment, 2), 30]}
    # Add Medical Debt
    elif debtType == 1:
        if debtor[2] > risk:
            cost = random.randint(200, 100000)
            initial_dp = random.randint(0,5000)
            interest_rate = random.randint(5,20) - (initial_dp / 1000)
            monthlyPayment = cost / random.choice([42, 60, 72]) * (interest_rate/100)
            debtObj = {f"Medical{id}": [cost, initial_dp, round(interest_rate,2), round(monthlyPayment, 2), 30]}
    # Add A Credit Card
    elif debtType == 2:
        if debtor[2] > risk:
            cost = random.randint(100, 500) + (10 * debtor[3])
            initial_dp = random.randint(0,50)
            interest_rate = random.randint(3,10) - (initial_dp / 1000)
            monthlyPayment = cost / random.choice([12, 16, 20]) * (interest_rate/100)
            debtObj = {f"Credit Card{id}": [cost, initial_dp, round(interest_rate,2), round(monthlyPayment, 2), 30]}
    # Add Personal Loan
    else:
        if debtor[2] > risk:
            cost = random.randint(1000, 50000) + (1000 * debtor[3])
            initial_dp = random.randint(0,5000)
            interest_rate = random.randint(0,20) - (initial_dp / 1000)
            monthlyPayment = cost / random.choice([12, 42, 60, 72]) * (interest_rate/100)
            debtObj = {f"Personal Loan{id}": [cost, initial_dp, round(interest_rate,2), round(monthlyPayment, 2), 30]}
        
    return debtObj

In [ ]:

def gameLoop(player, playerOwns):
    debtor = player
    playerOwned = playerOwns
    totalDebtLine = []
    numMonthsAlive = 600
    intrestDebtAccrued = 0
    
    for month in range(numMonthsAlive):
        # Geerating New Debt ensures that the models aren't biased due to Same Pattern of Debt
        if month % 100 == 0:
            debtor[4] += 10000
        debtor[3] = 1*(.1**len(playerOwned))
        objNew = genNewDebt(debtor)
        if not objNew == None:
            playerOwns.update(objNew)
        paid = 0
        '''
        # Implementation of a Simple First Come First Serve Scheduling
        COMPLETED!
        '''
        if debtor[5]=='fcfs':
            while(paid != player[4] and len(playerOwned) > 0):
                firstCome = next(iter(playerOwned))
                if playerOwned[firstCome][0] <= debtor[4]-paid:
                    paid += playerOwned[firstCome][0]
                    del playerOwned[firstCome]
                elif playerOwned[firstCome][0] >= debtor[4]-paid:
                    playerOwned[firstCome][0] -= debtor[4]-paid
                    if playerOwned[firstCome][3] <= debtor[4]-paid:
                        playerOwned[firstCome][4] = 30
                    paid = debtor[4]
        '''
        # Implementation of a Simple Shortest Job First (Least amount to payoff) Scheduling
        COMPLETED!
        '''
        if debtor[5]=='sjf':
            while(paid != player[4] and len(playerOwned) > 0):
                minJob = ''
                minJobSize = 100000000000
                for debt in playerOwned:
                    if playerOwned[debt][0] < minJobSize:
                        minJob = debt
                        minJobSize = playerOwned[debt][0]
                if playerOwned[minJob][0] <= debtor[4]-paid:
                    paid += playerOwned[minJob][0]
                    del playerOwned[minJob]
                elif playerOwned[minJob][0] >= debtor[4]-paid:
                    playerOwned[minJob][0] -= debtor[4]-paid
                    if playerOwned[minJob][3] <= debtor[4]-paid:
                        playerOwned[minJob][4] = 30
                    paid = debtor[4]
        '''
        # Implementation of a Simple Priority (Priority based on most pressing due date) Scheduling
        COMPLETED!
        '''
        if debtor[5]=='priority':
            while(paid != player[4] and len(playerOwned) > 0):
                minJob = ''
                minJobSize = 100
                
                for dueDate in playerOwned:
                    if playerOwned[dueDate][-1] < minJobSize:
                        minJob = dueDate
                        minJobSize = playerOwned[dueDate][-1]
                if playerOwned[minJob][0] <= debtor[4]-paid:
                    paid += playerOwned[minJob][0]
                    del playerOwned[minJob]
                elif playerOwned[minJob][0] >= debtor[4]-paid:
                    playerOwned[minJob][0] -= debtor[4]-paid
                    if playerOwned[minJob][3] <= debtor[4]-paid:
                        playerOwned[minJob][4] = 31
                    paid = debtor[4]

        '''
        # Implementation of a (Poor) Round Robin (all accounts get paid the same) Scheduling
        COMPLETED!
        '''
        if debtor[5]=='equal':
            accounts = len(playerOwned)
            equalPay = debtor[4]/accounts
            for it in playerOwned:
                playerOwned[it][0] -= equalPay
            
            paid = debtor[4]


        monthDebt = 0
        for itemowned in playerOwned:
            playerOwned[itemowned][-1] -= 1
            #Add interest Every single month and add it again if there is a missed payment
            intrestDebtAccrued += playerOwned[itemowned][3] * (playerOwned[itemowned][3] / 100)
            playerOwned[itemowned][0] += playerOwned[itemowned][3] * (playerOwned[itemowned][3] / 100)
            if playerOwned[itemowned][-1] < 0:
                intrestDebtAccrued += playerOwned[itemowned][3] * (playerOwned[itemowned][3] / 100)
                playerOwned[itemowned][0] += playerOwned[itemowned][3] * (playerOwned[itemowned][3] / 100)
            playerOwned[itemowned][0] = round(playerOwned[itemowned][0],2)
            
            monthDebt += playerOwned[itemowned][0]
        
        totalDebtLine.append(max(monthDebt,0))


    return totalDebtLine, intrestDebtAccrued   


## This is the part that runs the above methods
If you want to change the scheduler being used simply change it in the player list below. It will be the last list index.

There are 4 options:
- 'fcfs' = First come first serve - keep paying on first item in the list until it's payed off
- 'sjf' = Shortest Job First - Find the account with lowest amount and pay towards that one.
- 'priority' = Priority - Each account has to be payed in a count of 30 or intrest really piles on. So prioritize the ones getting close to that mark!
- 'equal' = Simply divide the players money equally among each account (Sorta Round Robin)

Changing both the age player[3] and income player[4] will effect the way the program run. Feel free to experiment.

In [ ]:
import copy



player = ['b', '24', .01, 24, 5000, 'equal']
playerOwns = {'House': [270000, 14000, 5.6, 500, 30], 'Car': [32000, 4000, 7.8, 344, 30]}


hund_Table = []
total_Table = []
total_Tax = []


for i in range(10000):
    totalDebt = 0
    intrestcost = 0
    debtor = copy.deepcopy(player)
    playerOwned = copy.deepcopy(playerOwns)
    totalDebt, intrestcost = gameLoop(debtor, playerOwned)
    
    if len(total_Table) == 0:
        for j in range(len(totalDebt)):
            total_Table.append(totalDebt[j])
    
    for l in range(len(totalDebt)):
        total_Table[l] += totalDebt[l]
        total_Table[l] = total_Table[l] / 2
    if i % 100 == 0:
        hund_Table.append(totalDebt)
    total_Tax.append(intrestcost) 



In [ ]:


plt.figure(figsize=(8,5))
plt.ticklabel_format(style='plain')
plt.title('Total Debt over A Lifetime 10000 Iterations Rand 100')
plt.ylabel('Total Debt')
plt.xlabel('Months')
for en in hund_Table:
    plt.plot(en)
plt.show()

plt.figure(figsize=(8,5))
plt.ticklabel_format(style='plain')
plt.title('Total Debt over A Lifetime 10000 Iterations Avg')
plt.ylabel('Total Debt')
plt.xlabel('Months')
plt.plot(total_Table)
plt.show()


print("Total Debt gained from intreset",round(sum(total_Tax)/len(total_Tax),2))